In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset/gemstone.csv')

df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df.drop('id', axis=1, inplace=True)

In [4]:
# independent and dependent features
X = df.drop('price', axis=1)
y = df[['price']]

In [5]:
# defined the custom ranking of each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

In [31]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   carat    193573 non-null  float64
 1   cut      193573 non-null  object 
 2   color    193573 non-null  object 
 3   clarity  193573 non-null  object 
 4   depth    193573 non-null  float64
 5   table    193573 non-null  float64
 6   x        193573 non-null  float64
 7   y        193573 non-null  float64
 8   z        193573 non-null  float64
dtypes: float64(6), object(3)
memory usage: 13.3+ MB


In [36]:
num_columns = X.columns[X.dtypes != 'object']
cat_columns = X.columns[X.dtypes == 'object']

* automating

In [11]:
from sklearn.impute import SimpleImputer # to handle missing values
from sklearn.preprocessing import StandardScaler # for feature scaling
from sklearn.preprocessing import OrdinalEncoder

# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [39]:
# Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

# Categorical Pipleline
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
        ('scaler', StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, num_columns),
    ('cat_pipeline', cat_pipeline, cat_columns)
])

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [42]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())

In [44]:
# Model Training

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [45]:
reg = LinearRegression().fit(X_train, y_train)

In [46]:
reg.coef_

array([[ 6430.67767479,  -131.82597587,   -71.57263787, -1756.49729491,
         -430.33020445,   -87.5346621 ,    67.85347115,  -464.01098494,
          651.10737279]])

In [47]:
reg.intercept_

array([3972.75199581])

In [59]:
import numpy as np

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [66]:
# Training multiple models

models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge':Ridge(),
    'ElasticNet': ElasticNet()
}

trained_model_list = []
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    #prediction
    y_pred = model.predict(X_test)
    mae, mse, rmse, r2_square = evaluate_model(y_test, y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print('RMSE: ', rmse)
    print('MAE: ', mae)
    print('MSE: ', mse)
    print('R2 Score: ', r2_square * 100)

    r2_list.append(r2_square)

    print('=' * 35)
    print('\n')
    


LinearRegression
Model Training Performance
RMSE:  1018.1581640245504
MAE:  675.8646403365891
MSE:  1036646.0469698432
R2 Score:  93.61253831282488


Lasso
Model Training Performance
RMSE:  1018.2756930699602
MAE:  677.0721490787183
MSE:  1036885.3870971077
R2 Score:  93.61106358005802


Ridge
Model Training Performance
RMSE:  1018.1611429115786
MAE:  675.8935091356298
MSE:  1036652.1129350121
R2 Score:  93.61250093640288


ElasticNet
Model Training Performance
RMSE:  1538.6866758139975
MAE:  1062.8025492504892
MSE:  2367556.6863275296
R2 Score:  85.41191791514899




In [67]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']